In [ ]:
# import matplotlib.pyplot as plt
from datetime  import datetime, timedelta
import numpy as np
import pandas as pd
import Nio
import json
import gc
import pycurl
import xarray as xr
import time, random, glob, shutil, os
import re
import urllib
import http.client
import certifi
from concurrent.futures import ThreadPoolExecutor, as_completed

try:
    from io import BytesIO
except ImportError:
    from StringIO import StringIO as BytesIO


class model(object):

    def __init__(self):

        """ 
            Função para inicializar o configurador do modelo BAM, retorna objeto com a função load habilitada para uso.

            Parametros
            ------------------------------------------------------------------------------------------------------------------------------------------------------       

            * Model     : Para configurar o BAM em novas resoluções altere o campo parameter
            * Variables : Para habilitar novas váriaveis adicionar o nome da variavel e o nome referente dentro do .idx ou .inv
            * Levels    : Define as variaveis com 1 unico nivel ou multiplos
            * Area      : Durante inicialização é adicionado o campo Reduce, quando True os parametros definidos aqui são aplicados para dar zoom em area desejada
            * Transform : Realiza transformação nas unidades das variaveis para uma unidade comum entre os modelos.
            * File      : Nome do arquivo disponivel no ftp
            * Server    : Servidor FTP consumido pela aplicação
            ------------------------------------------------------------------------------------------------------------------------------------------------------       

            Retorna objeto model
        """
        
        self.dict = {   
                    "model"     : {
                                    "name" : "bam",
                                    "parameter" : "TQ0666L064",
                                    "long_name" : "The Brazilian Global Atmospheric Model"
                                },
                    "variables" :  {  
                                    "t" : "temp",
                                    "u" : "uvel",
                                    "v" : "vvel",
                                    "rh" : "umrl",
                                    "g" : "zgeo",
                                    "omega" : "omeg",
                                    "u10m" : "uves",
                                    "v10m" : "vves",
                                    "t2m" : "tp2m",
                                    "slp" : "psnm",
                                    "psfc" : "pslc",
                                    "terrain" : "topo",
                                    "sbcape" : "cape",
                                    "sbcin" : "cine",
                                    "pw" : "agpl",
                                    "precip" : "prec"
                                },
                    "levels" :   {  
                                    "t" : "LVL",
                                    "u" : "LVL",
                                    "v" : "LVL",
                                    "rh" : "LVL",
                                    "g" : "LVL",
                                    "omega" : "LVL",
                                    "u10m" : "SFC",
                                    "v10m" : "SFC",
                                    "t2m" : "SFC",
                                    "slp" : "SFC",
                                    "psfc" : "SFC",
                                    "terrain" : "SFC",
                                    "sbcape" : "SFC",
                                    "sbcin" : "SFC",
                                    "pw" : "SFC",
                                    "precip" : "SFC"
                                },

                    "area"    : {
                                    "minlat" :    -45,
                                    "maxlat" :     10,
                                    "minlon" :    277,
                                    "maxlon" :    332,
                                    "dx"     :  20000
                                },

                    "transform" : {
                                    "precip" :      "/4",
                                    "t"      : "-273.15",
                                    "rh"     :    "*100",
                                    "omega"  :     "*10",
                                    "t2m"    :  "-273.15"
                                },

                    "file"    : {
                                    "name"   :     "GPOSNMC{}{}P.fct.TQ0666L064.grb",
                                    "format" :     "grb"
                                },
            "server":   {
                            "ftp"    :     "https://dataserver.cptec.inpe.br"
            }
    
        } 

        self.levels=  ["1000","925","850","775","700","500","400","300","250","200","150","100","70","50","30","20","10","3"]

        self.dict['area'].update({'reduce': False})
        self.dict.update({'save_netcdf': False})
        self.dict.update({'path_to_save': os.getcwd()})
        self.dict.update({'lon_360_to_180': False})

        self.local_path = f"INPE/{self.dict['model']['name']}/{self.dict['model']['parameter']}/brutos"
        self.ftppath = f"/dataserver_modelos/{self.dict['model']['name']}/{self.dict['model']['parameter']}/brutos"
        
        print(f"\n#### {self.dict['model']['long_name']} ({self.dict['model']['parameter']} / Hybrid) #####\n")
        start = time.strftime("%Y%m%d", time.gmtime(time.time()))
        end = (datetime.strptime(f'{start}',  '%Y%m%d') - timedelta(days=10)).strftime("%Y%m%d")
        self.daterange =  pd.date_range(end, start)
        self.frequency = [0,28]
        self.area = {
                       "northlat" :    90,
                       "southlat" :    -90,
                       "westlon" :    0,
                       "eastlon" :    360,
                       "invertedlat" : False
                    }

        print(f"Forecast data available for reading between {end} and {start}.\n")
        print(f"Surface variables: t2m, u10m, v10m, slp, psfc, precip")
        print(f"                   terrain, sbcape, sbcin, pw.")
        print(f"Level variables:   t, u, v, rh, g, omega.\n")
        print(f"levels (hPa): 1000  925  850  775  700  500  400  300  250")
        print(f"              200 150  100   70   50   30   20   10    3.\n")
        print(f"Frequency: every 6 hours [0, 6, 12, 18,...,168].\n")
        self.session = random.random()
        model.__clean__()
  

    def load(self, date=None, steps=[0], var=['t2m'], level=[1000, 'sfc']):

        """
        
        A função load prepara a lista de variaveis, niveis e datas que serão carregadas para memoria.

        Durante execução um diretorio temporario é criado para manipular os arquivos e é apagado assim que finalizada requisição.

        self.date é definido pela frequência que o modelo disponibiliza suas previsões, para o BAM de 6 em 6 horas.
        
        Parametros
        ------------------------------------------------------------------------------------------------------------       
        date  : Data da condição inicial date=YYYYMMDDHH, use HH para IC 00 e 12.
        steps : Array de inteiros com os passos desejados. onde 0 é a inicialização do modelo [0,1, ... ,28], valor maximo 28.
        var   : Array de string com nome das variaveis disponiveis para leitura ['t2m', 'precip']
        level : Array de inteiros com os niveis disponiveis para cada modelo [1000, 850]
        ------------------------------------------------------------------------------------------------------------       

        load(date='2022082300', steps=[0,1,5,9], var=['t', 'precip'], level=[1000, 850])

        ------------------------------------------------------------------------------------------------------------       
        
        Retorna um Xarray contendo todas variaveis solicitadas com as transformações contidas em self.dict

        ------------------------------------------------------------------------------------------------------------       

        """
        if (isinstance(steps,int)) : steps = [h for h in range(0, steps+1, 1)]
        if type(date) == int: date = str(date)
        if date == None: date = datetime.today().strftime("%Y%m%d")

        if type(level) == int: level = [level]
        if type(var) == str: var = [var]

        self.start_date = date
        self.start_date = self.start_date.replace('/', '')
        self.start_date = self.start_date.replace('-', '')

        if len(self.start_date) == 8: self.start_date = f"{self.start_date}00"

        self.query_level = level
        self.date = [(datetime.strptime(f'{self.start_date}',  '%Y%m%d%H') + timedelta(hours=int(h*6))).strftime("%Y%m%d%H") for h in steps]
        self.year       = self.start_date[0:4]
        self.mon        = self.start_date[4:6]
        self.day        = self.start_date[6:8]
        self.hour       = self.start_date[8:10]

        self.variables = var

        self.__getrange__()
        if os.path.exists(f".temporary_files/{self.session}"): shutil.rmtree(f".temporary_files/{self.session}")
        
        return self.file

    def __repr__(self):

        """
            Função para exibir definições contidas no objeto, acessivel através do self.dict

        """

        print(f"Reduce area: {self.dict['area']['reduce']}")
        print(f"Save netcdf: {self.dict['save_netcdf']}")
        print(f"Path to save: {self.dict['path_to_save']}")
        print(f"To see more info use bam.help()")

        return str('')    

    def __clean__():

        """
            Quando o processo de requisição é interrompido a ferramenta não removerá os arquivos temporarios,
            esta função remove todo diretorio temporario com mais de 2 dias em disco.

        """
        
        if os.path.exists(f".temporary_files"): 

            today = datetime.today()
            
            files = glob.glob(".temporary_files/0.*")
            for f in files:
                duration = today - datetime.fromtimestamp(os.path.getmtime(f))
                if duration.days >= 2:
                    shutil.rmtree(f)
    
    def help(self):

        """
            Função para exibir as informações dos modelos e suas parametrizações.
        
        """
        
        print('help')

    def __getrange__(self):

        """ 
            Função para criar dataframe com informações que serão consumidas por self.__curl__.
            Entre as informações coletadas estão as posições inferior e superior de cada variavel dentro no arquivo grib.

            Exemplo self.setup:
            --------------------------------------------------------------------------------------------------------------       
                forecast_date      upper   id      lower  start_date   var          level step_model varname
            0   2022082300  780016380  195  776016296  2022082300  tp2m  2 m above gnd        anl     t2m
            1   2022082306  780016380  195  776016296  2022082300  tp2m  2 m above gnd        anl     t2m
            --------------------------------------------------------------------------------------------------------------       

        """

        arr = []

        try:

            for dt in self.date:

                time.sleep(1)

                invfile = self.dict['file']['name'].format(self.start_date, dt)
                invfile = invfile.split('.grb')[:-1]

                invfile = f'{self.ftppath}/{self.year}/{self.mon}/{self.day}/{self.hour}/{invfile[0]}.inv'
                #print(f"{self.dict['server']['ftp']}/{invfile}")

                while True:
                    try:
                        # print(f"{self.dict['server']['ftp']}/{invfile}")
                        df = pd.read_csv(f"{self.dict['server']['ftp']}/{invfile}", skiprows=0, names=['header'])
                        break
                    except urllib.error.HTTPError as err:
                        print('File not available on server!',err)
                        time.sleep(1)
                        continue
                        # self.file = None
                    except http.client.IncompleteRead as err:
                        print("Retrying pd.read_csv..")
                        time.sleep(1)
                        continue
                df['header'] = df['header'].map(lambda x: x[:-1])
                df[['id', 'allocate', 'date', 'var','kpds5','kpds6','kpds7','tr', 'p1','p2', 'timeu','level', 'timeFct', 'nave']] = df['header'].str.split(':', expand=True)
                df.drop('header', axis=1, inplace=True)

                df['date'] = df['date'].map(lambda x: str(x).split('=')[1])

                for var in self.variables:
                    if var in self.dict['variables']:

                        value = self.dict['variables'][var]

                        varframe = df[ df['var'] == value ].copy()
                        varframe.drop(['kpds5','kpds6','kpds7','tr', 'p1', 'p2', 'timeu', 'nave'], axis=1, inplace=True)


                        # Add 1000 and surface when not defined on request
                        tmp_list = [i for i in self.query_level if i != 'surface']
                        if self.dict['levels'][var] == 'LVL' and len(tmp_list) == 0:
                            self.query_level.append(1000)

                        tmp_list = [i for i in self.query_level if i == 'surface']
                        if self.dict['levels'][var] == 'SFC' and len(tmp_list) == 0:
                            self.query_level.append('sfc')

                        for lvl in self.query_level:

                            if self.dict['levels'][var] == 'LVL' and lvl == 'sfc':
                                pass
                            elif self.dict['levels'][var] == 'SFC' and lvl != 'sfc':
                                pass
                            else:

                                if lvl == 'sfc': 
                                    
                                    if var == 't2m': lvl = '2 m above gnd'

                                    if var == 'topo': lvl = 'sfc'

                                    if var == 'slp' or value == 'psnm': lvl = 'MSL'

                                    if var == 'v10m' or var == 'u10m': lvl = '10 m above gnd'

                                    if var == 'pw': lvl = 'atmos'

                                if len(varframe) == 1:

                                    frame = varframe

                                else:

                                    if self.dict['levels'][var] == 'SFC':
                                
                                        frame = varframe[ varframe['level'] == 'sfc' ].copy()
                                    
                                    else:

                                        frame = varframe[ varframe['level'] == f'{lvl} mb' ].copy()

                                frame['date'] = self.start_date
                                upper = df.iloc[frame.index+1]['allocate']
                                pp = np.append(dt, upper)

                                try:
                                    pp = np.append(pp, frame.values[0])
                                except IndexError:
                                    pp = np.append(pp, frame.values)
                                
                                pp = np.append(pp, var)

                                if pp.shape[0] == 9:
                                    arr.append(pp)
                
            self.setup = pd.DataFrame(arr, columns=['forecast_date', 'upper', 'id',
                                                    'lower', 'start_date', 'var', 
                                                    'level', 'step_model', 'varname'])

            self.setup.drop_duplicates(inplace=True)
            self.__curl__()
            
        except urllib.error.HTTPError as err:
            print('File not available on server!', err)
            self.file = None
            return
        # except Exception as err:
        #     print(err)
        #     print(f"Unexpected {err=}, {type(err)=}")

    def baixar_grib(self, row, pathout):
        c = pycurl.Curl()  # Cada thread cria seu próprio objeto Curl!
        while True:
            try:
                grbfile = self.dict['file']['name'].format(row['start_date'], row['forecast_date'])
                grbfile = f"{self.ftppath}/{self.year}/{self.mon}/{self.day}/{self.hour}/{grbfile}"
                c.setopt(pycurl.URL, f"{self.dict['server']['ftp']}/{grbfile}")
                outfile = self.dict['file']['name'].format(row['start_date'], row['forecast_date'])
                lvl = row['level'].replace(" ", "")
                outfile = f"{pathout}/{row['varname']}_{lvl}_{outfile}"
                with open(outfile, "wb") as fout:
                    c.setopt(pycurl.WRITEDATA, fout)
                    c.setopt(c.RANGE, f"{row['lower']}-{row['upper']}")
                    c.setopt(pycurl.VERBOSE, 0)
                    c.setopt(pycurl.FOLLOWLOCATION, 0)
                    c.setopt(pycurl.TIMEOUT, 60)
                    c.setopt(pycurl.CAINFO, certifi.where())
                    c.perform()
                    response_code = c.getinfo(pycurl.RESPONSE_CODE)
                c.reset()
                if response_code != http.client.PARTIAL_CONTENT:
                    raise pycurl.error(f"Error with http response {response_code}")
                return outfile, row
            
            except http.client.IncompleteRead as err:
                print("Incomplete read. Retrying pycurl..",err)
                time.sleep(1)
                continue
            except pycurl.error as err:
                print(err)
                print("Retrying pycurl..")
                time.sleep(1)
                continue
            # except Exception as err:
            #     print(f"Erro no download de {row['varname']} {row['level']} {row['forecast_date']}: {err}")
            #     tries += 1
            #     time.sleep(0.5)
        # print(f"Falha no download após {max_tries} tentativas para {row['varname']} {row['level']} {row['forecast_date']}")
        # return None

    def __curl__(self):

        """
        
            A função __curl__ realiza para cada registro em self.setup o download das variaveis descritas, aplica as transformações
            definidas em self.dict['transform'] e devolve em self.file um Xarray em memoria com todos tempos das variaveis previstas solicitas.

            Quando self.dict['save_netcdf'] == True um arquivo netcdf4 será salvo com copia da requisição automaticamente.
        
        """

        pathout = f".temporary_files/{self.session}"

        os.makedirs(pathout, exist_ok=True)
        
        # removing old idx when running more than once
        fidx = glob.glob(f"{pathout}/*.idx")
        if len(fidx) > 0:
            [os.remove(f) for f in fidx]
        
        c = pycurl.Curl()

        rows = [row for _, row in self.setup.iterrows()]
        n_threads = 20

        with ThreadPoolExecutor(max_workers=n_threads) as executor:
            futures = [executor.submit(self.baixar_grib, row, pathout) for row in rows]
            outfiles_rows = []
            for future in as_completed(futures):
                result = future.result()
                if result:
                    outfiles_rows.append(result)
                    # print(f"Arquivo baixado: {result}")

        for (outfile, row) in outfiles_rows:
            try:
                f = xr.open_dataset(outfile, engine='pynio')
            except Nio.NIOError as err:
                print(f"Open Dataset Error: {err}")
                continue

            f = f.assign_coords({'time': datetime.strptime(row['forecast_date'],  '%Y%m%d%H')})
            f = f.rename({'g0_lat_0' : 'latitude'})
            f = f.rename({'g0_lon_1' : 'longitude'})
            if self.dict['lon_360_to_180'] == True:
                f = f.assign_coords(longitude=((f.longitude + 180) % 360 - 180))
                f = f.reindex({"longitude": np.sort(f.longitude.values)})

            v = list(f.keys())
            var = outfile.split('/')[-1]
            var = var.split('_')[0]
            f = f.rename({v[0] : var})

            if 'mb' in row['level']:
                lev = re.sub("[^0-9]", "", row['level'])
                f = f.assign_coords({'level': float(lev)})
                f = f.expand_dims(['level'])
                f.level.attrs = {
                            "long_name" :	 "pressure",
                            "units"  :       "hPa",
                            "positive":      "down",
                            "standard_name": "air_pressure"
        }

            f = f.expand_dims(['time'])
            outnc = outfile.split('/')[-1]
            outnc = outnc.split('.')[:-1]
            outnc = '.'.join(str(e) for e in outnc)

            # Transform variables in self.dict['transform']
            if var in self.dict['transform']:
                tr = float(self.dict['transform'][var][1:])
                op = self.dict['transform'][var][0]
                f = eval(f'f {op} tr')

            if 't2m' in f:
                f['t2m'].attrs = {
                            "long_name" :	 "Surface Temperature",
                            "units"  :       "C",
                            "standard_name": "temperature"
        }

            if 't' in f:
                f['t'].attrs = {
                            "long_name" :	 "Temperature",
                            "units"  :       "C",
                            "standard_name": "temperature"
        }
            if os.path.exists(f"{pathout}/{outnc}.nc4"): os.remove(f"{pathout}/{outnc}.nc4")

            if self.dict['area']['reduce'] ==  True:

                lat1 = self.dict['area']['minlat']
                lat2 = self.dict['area']['maxlat']
                lon1 = self.dict['area']['minlon']
                lon2 = self.dict['area']['maxlon']
            
                f2 = f.sel(latitude=slice(lat2, lat1), 
                        longitude=slice(lon1, lon2)).copy()

            else:

                f2 = f

            f2.to_netcdf(f'{pathout}/{outnc}.nc4', encoding={'time': {'dtype': 'i4'}})
            f2.close()
            f.close()

            gc.collect()

        c.close()

        files = glob.glob(f"{pathout}/*.nc4")

        if len(files) == 1:
            fout = xr.open_dataset(files[0], chunks={'latitude': 150, 'longitude': 150})

        else:    
            fout = xr.open_mfdataset(files,  combine='nested', parallel=False,  chunks={'latitude': 1500, 'longitude': 1500})

        fout.attrs = {
                            "center" :	"National Institute for Space Research - INPE",
                            "model"  :  f"The Brazilian Global Atmospheric Model ({self.dict['model']['parameter']} / Hybrid)"
        }
        fout.time.encoding['units'] = "Seconds since 1970-01-01 00:00:00"

        if self.dict['save_netcdf'] == True:

            pathout = f"{self.dict['path_to_save']}/{self.local_path}/{self.year}/{self.mon}/{self.day}/{self.hour}"
            os.makedirs(pathout, exist_ok=True)
            ncout = self.dict['file']['name'].format(row['start_date'], row['forecast_date'])
            ncout = ncout.replace(f"{self.dict['file']['format']}","nc4")

            if os.path.exists(f"{pathout}/{ncout}"): os.remove(f"{pathout}/{ncout}")
            encoding = {var: {'zlib': True, 'complevel': 1, 'shuffle': True} for var in fout.data_vars}
            fout.to_netcdf(f"{pathout}/{ncout}", format="NETCDF4", encoding=encoding)

        field = fout.load()
        fout.close()
        del fout

        gc.collect()
        self.file =  field


In [3]:
# import cptecmodel.CPTEC_BAM as BAM
# from datetime import datetime, timedelta

bam = model()

date = '2022111800'

# Variaveis 
vars = ['t2m']

# Nive                                      is
levels = [1000]

# Steps = Numero de simulações futuras a partir da inicialização do modelo
steps = 4

bam.dict['area']['reduce'] = True
bam.dict['area']['minlat'] = -16.0
bam.dict['area']['maxlat'] = -15.6
bam.dict['area']['minlon'] = 311.7
bam.dict['area']['maxlon'] = 312.3

f = bam.load(date=date, var=vars,level=levels, steps=steps)
f


#### The Brazilian Global Atmospheric Model (TQ0666L064 / Hybrid) #####

Forecast data available for reading between 20250602 and 20250612.

Surface variables: t2m, u10m, v10m, slp, psfc, precip
                   terrain, sbcape, sbcin, pw.
Level variables:   t, u, v, rh, g, omega.

levels (hPa): 1000  925  850  775  700  500  400  300  250
              200 150  100   70   50   30   20   10    3.

Frequency: every 6 hours [0, 6, 12, 18,...,168].

https://dataserver.cptec.inpe.br//dataserver_modelos/bam/TQ0666L064/brutos/2022/11/18/00/GPOSNMC20221118002022111800P.fct.TQ0666L064.inv
https://dataserver.cptec.inpe.br//dataserver_modelos/bam/TQ0666L064/brutos/2022/11/18/00/GPOSNMC20221118002022111806P.fct.TQ0666L064.inv
https://dataserver.cptec.inpe.br//dataserver_modelos/bam/TQ0666L064/brutos/2022/11/18/00/GPOSNMC20221118002022111812P.fct.TQ0666L064.inv
https://dataserver.cptec.inpe.br//dataserver_modelos/bam/TQ0666L064/brutos/2022/11/18/00/GPOSNMC20221118002022111818P.fct.TQ0666L064.in

*   Trying 150.163.178.23:443...
* Connected to dataserver.cptec.inpe.br (150.163.178.23) port 443 (#0)
* ALPN, offering h2
* ALPN, offering http/1.1
* successfully set certificate verify locations:
*  CAfile: /home/juliababa/miniconda3/envs/py38/lib/python3.8/site-packages/certifi/cacert.pem
*  CApath: none
* SSL connection using TLSv1.3 / TLS_AES_256_GCM_SHA384
* ALPN, server accepted to use h2
* Server certificate:
*  subject: CN=dataserver.cptec.inpe.br
*  start date: May 21 00:43:25 2025 GMT
*  expire date: Aug 19 00:43:24 2025 GMT
*  subjectAltName: host "dataserver.cptec.inpe.br" matched cert's "dataserver.cptec.inpe.br"
*  issuer: C=US; O=Let's Encrypt; CN=R11
*  SSL certificate verify ok.
* Using HTTP2, server supports multiplexing
* Connection state changed (HTTP/2 confirmed)
* Copying HTTP/2 data in stream buffer to connection buffer after upgrade: len=0
* Using Stream ID: 1 (easy handle 0x624a61bef3c0)
> GET //dataserver_modelos/bam/TQ0666L064/brutos/2022/11/18/00/GPOSNMC20

https://dataserver.cptec.inpe.br//dataserver_modelos/bam/TQ0666L064/brutos/2022/11/18/00/GPOSNMC20221118002022111806P.fct.TQ0666L064.grb


* Found bundle for host dataserver.cptec.inpe.br: 0x624a61c028b0 [can multiplex]
* Re-using existing connection! (#0) with host dataserver.cptec.inpe.br
* Connected to dataserver.cptec.inpe.br (150.163.178.23) port 443 (#0)
* Using Stream ID: 3 (easy handle 0x624a61bef3c0)
> GET //dataserver_modelos/bam/TQ0666L064/brutos/2022/11/18/00/GPOSNMC20221118002022111806P.fct.TQ0666L064.grb HTTP/2
Host: dataserver.cptec.inpe.br
range: bytes=1376028896-1380028980
user-agent: PycURL/7.44.1 libcurl/7.79.1 OpenSSL/1.1.1w zlib/1.2.13 libssh2/1.10.0 nghttp2/1.43.0
accept: */*

< HTTP/2 206 
< date: Thu, 12 Jun 2025 12:34:34 GMT
< server: gunicorn
< x-frame-options: DENY
< x-xss-protection: 1; mode=block
< x-content-type-options: nosniff
< content-disposition: inline; filename=GPOSNMC20221118002022111806P.fct.TQ0666L064.grb
< content-type: application/octet-stream
< content-length: 4000085
< last-modified: Tue, 28 Mar 2023 17:57:55 GMT
< cache-control: no-cache
< etag: "1680026275.003509-1696040908-12

https://dataserver.cptec.inpe.br//dataserver_modelos/bam/TQ0666L064/brutos/2022/11/18/00/GPOSNMC20221118002022111812P.fct.TQ0666L064.grb


...

https://dataserver.cptec.inpe.br//dataserver_modelos/bam/TQ0666L064/brutos/2022/11/18/00/GPOSNMC20221118002022111900P.fct.TQ0666L064.grb


* Found bundle for host dataserver.cptec.inpe.br: 0x624a61c028b0 [can multiplex]
* Re-using existing connection! (#0) with host dataserver.cptec.inpe.br
* Connected to dataserver.cptec.inpe.br (150.163.178.23) port 443 (#0)
* Using Stream ID: 9 (easy handle 0x624a61bef3c0)
> GET //dataserver_modelos/bam/TQ0666L064/brutos/2022/11/18/00/GPOSNMC20221118002022111900P.fct.TQ0666L064.grb HTTP/2
Host: dataserver.cptec.inpe.br
range: bytes=1376028896-1380028980
user-agent: PycURL/7.44.1 libcurl/7.79.1 OpenSSL/1.1.1w zlib/1.2.13 libssh2/1.10.0 nghttp2/1.43.0
accept: */*

< HTTP/2 502 
< date: Thu, 12 Jun 2025 12:35:07 GMT
< server: Apache
< x-frame-options: DENY
< x-xss-protection: 1; mode=block
< x-content-type-options: nosniff
< content-length: 341
< content-type: text/html; charset=iso-8859-1
< 
* Connection #0 to host dataserver.cptec.inpe.br left intact


Error with http response 502
Retrying pycurl..


* Found bundle for host dataserver.cptec.inpe.br: 0x624a61c028b0 [can multiplex]
* Re-using existing connection! (#0) with host dataserver.cptec.inpe.br
* Connected to dataserver.cptec.inpe.br (150.163.178.23) port 443 (#0)
* Using Stream ID: b (easy handle 0x624a61bef3c0)
> GET //dataserver_modelos/bam/TQ0666L064/brutos/2022/11/18/00/GPOSNMC20221118002022111900P.fct.TQ0666L064.grb HTTP/2
Host: dataserver.cptec.inpe.br
range: bytes=1376028896-1380028980
user-agent: PycURL/7.44.1 libcurl/7.79.1 OpenSSL/1.1.1w zlib/1.2.13 libssh2/1.10.0 nghttp2/1.43.0
accept: */*

< HTTP/2 206 
< date: Thu, 12 Jun 2025 12:35:46 GMT
< server: gunicorn
< x-frame-options: DENY
< x-xss-protection: 1; mode=block
< x-content-type-options: nosniff
< content-disposition: inline; filename=GPOSNMC20221118002022111900P.fct.TQ0666L064.grb
< content-type: application/octet-stream
< content-length: 4000085
< last-modified: Tue, 28 Mar 2023 18:01:45 GMT
< cache-control: no-cache
< etag: "1680026505.7764-1696040908-1240

<xarray.Dataset>
Dimensions:    (latitude: 2, longitude: 4, time: 5)
Coordinates:
  * latitude   (latitude) float32 -15.77 -15.95
  * longitude  (longitude) float32 311.8 311.9 312.1 312.3
  * time       (time) datetime64[ns] 2022-11-18 ... 2022-11-19
Data variables:
    t2m        (time, latitude, longitude) float32 18.94 18.79 ... 18.95 20.86
Attributes:
    center:   National Institute for Space Research - INPE
    model:    The Brazilian Global Atmospheric Model (TQ0666L064 / Hybrid)

In [3]:
# Carregue o dataset (exemplo)
# ds = xr.open_dataset("seu_arquivo.nc")

# Defina as coordenadas de Brasília
lat_brasilia = -15.8
lon_brasilia = -47.9

'2022111800'
data_brasilia = "2022-11-18T18:00:00"


# Seleção do dado mais próximo de Brasília e horário
# Ajuste o nome da variável ('temperature') para o correto do seu dataset
temp_brasilia = f['t2m'].sel(
    latitude=lat_brasilia, 
    longitude=lon_brasilia, 
    time=data_brasilia, 
    method="nearest"
).values.item()

print(f"Temperatura prevista para Brasília em {data_brasilia}: {temp_brasilia:.2f}")

f['t2m']


Temperatura prevista para Brasília em 2022-11-18T18:00:00: 28.77


<xarray.DataArray 't2m' (time: 5, latitude: 2, longitude: 4)>
array([[[18.938568, 18.788177, 20.155365, 21.674896],
        [17.932709, 18.206146, 20.372162, 21.26474 ]],

       [[19.409698, 19.087433, 18.089386, 17.71048 ],
        [21.315948, 20.313995, 19.259308, 19.524933]],

       [[22.623566, 22.373566, 22.147003, 21.564972],
        [22.656769, 21.953644, 21.709503, 21.789581]],

       [[28.766022, 29.035553, 28.787506, 28.635162],
        [28.428131, 28.676178, 28.225006, 28.57071 ]],

       [[21.514465, 19.553528, 19.94806 , 20.137512],
        [20.420715, 20.033997, 18.950012, 20.864075]]], dtype=float32)
Coordinates:
  * latitude   (latitude) float32 -15.77 -15.95
  * longitude  (longitude) float32 311.8 311.9 312.1 312.3
  * time       (time) datetime64[ns] 2022-11-18 ... 2022-11-19
Attributes:
    long_name:      Surface Temperature
    units:          C
    standard_name:  temperature

In [ ]:
import pandas as pd
from datetime import datetime

df = pd.read_csv('/home/juliababa/tcc/src/forecast_brasilia.csv', sep=';')

datas_modelo = pd.to_datetime(df['data']).dt.strftime('%Y%m%d00').unique()

bam = model()

print(datas_modelo)

bam.dict['area']['reduce'] = True
bam.dict['area']['minlat'] = -16.0
bam.dict['area']['maxlat'] = -15.6
bam.dict['area']['minlon'] = 311.7
bam.dict['area']['maxlon'] = 312.3

vars = ['t2m']
levels = [1000]
steps = 4

# Lista para armazenar todos os resultados
resultados = []

for date in datas_modelo:
    print(f"Solicitando para {date}")
    f = bam.load(date=date, var=vars, level=levels, steps=steps)
    # Certifique-se de que 'f' é um DataFrame, xarray.Dataset ou algo convertível
    if isinstance(f, pd.DataFrame):
        resultados.append(f)
    elif hasattr(f, "to_dataframe"):
        resultados.append(f.to_dataframe().reset_index())
    else:
        resultados.append(pd.DataFrame(f))

# Concatenar tudo em um único DataFrame
df_all_results = pd.concat(resultados, ignore_index=True)
print(df_all_results.head())

# (Opcional) Salvar em CSV
df_all_results.to_csv("bam_data.csv", sep=";", index=False)



#### The Brazilian Global Atmospheric Model (TQ0666L064 / Hybrid) #####

Forecast data available for reading between 20250602 and 20250612.

Surface variables: t2m, u10m, v10m, slp, psfc, precip
                   terrain, sbcape, sbcin, pw.
Level variables:   t, u, v, rh, g, omega.

levels (hPa): 1000  925  850  775  700  500  400  300  250
              200 150  100   70   50   30   20   10    3.

Frequency: every 6 hours [0, 6, 12, 18,...,168].

['2023102000' '2023102100' '2023102200' '2023102300' '2023102400'
 '2023102500' '2023102600' '2023102700' '2023102800' '2023102900'
 '2023103000' '2023103100' '2023110100' '2023110200' '2023110300'
 '2023110400' '2023110500' '2023110600' '2023110700' '2023110800'
 '2023110900' '2023111000' '2023111100' '2023111200' '2023111300'
 '2023111400' '2023111500' '2023111600' '2023111700' '2023111800'
 '2023111900' '2023112000' '2023112100' '2023112200' '2023112300'
 '2023112400' '2023112500' '2023112600' '2023112700' '2023112800'
 '2023112900' '202

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os

df = pd.read_csv('/home/juliababa/tcc/src/forecast_brasilia.csv', sep=';')
datas_modelo = pd.to_datetime(df['data']).dt.strftime('%Y%m%d00').unique()

bam = model()
bam.dict['area']['reduce'] = True
bam.dict['area']['minlat'] = -16.0
bam.dict['area']['maxlat'] = -15.6
bam.dict['area']['minlon'] = 311.7
bam.dict['area']['maxlon'] = 312.3

vars = ['t2m']
steps = 3

csv_path = "bam_data.csv"
primeiro = True  # Controle para o cabeçalho

# Coordenadas alvo
lat_alvo = -15.789343
lon_alvo = 312.074244

# Verifica se já existe arquivo de resultados
if os.path.isfile(csv_path):
    df_existente = pd.read_csv(csv_path, sep=";")
    # Assume que há uma coluna chamada "date" ou similar
    # Identifica a última data salva
    # Ajuste conforme o nome real da coluna (ex: 'time', 'date', etc)
    if 'time' in df_existente.columns:
        ultima_data_str = df_existente['time'].max()
    else:
        raise ValueError("Arquivo existente não tem coluna de data reconhecida ('date' ou 'time').")
    # Converte para datetime e para o formato '%Y%m%d00'
    ultima_data = pd.to_datetime(ultima_data_str)
    proxima_data = ultima_data + timedelta(days=1)
    proxima_data_str = proxima_data.strftime('%Y%m%d00')
    # Filtra datas para começar do dia seguinte
    datas_modelo = [d for d in datas_modelo if d >= proxima_data_str]
    primeiro = False  # Não escreve cabeçalho se já existe arquivo
    print(f"Iniciando a partir de {proxima_data_str}")
else:
    print("Nenhum arquivo anterior encontrado, começando do início.")

for date in datas_modelo:
    print(f"Solicitando para {date}")
    f = bam.load(date=date, var=vars, steps=steps)
    # Se for xarray.Dataset
    if hasattr(f, "sel"):
        # Encontra as coordenadas mais próximas
        lat_mais_proxima = float(f.latitude.sel(latitude=lat_alvo, method="nearest").values)
        lon_mais_proxima = float(f.longitude.sel(longitude=lon_alvo, method="nearest").values)
        # Seleciona o dado
        dado_ponto = f.sel(latitude=lat_mais_proxima, longitude=lon_mais_proxima)
        # Transforma em DataFrame (incluindo as coordenadas como colunas)
        df_res = dado_ponto.to_dataframe().reset_index()
    elif isinstance(f, pd.DataFrame):
        df_res = f
    else:
        df_res = pd.DataFrame(f)

    # Salva incrementalmente no CSV
    df_res.to_csv(
        csv_path,
        sep=";",
        index=False,
        mode='a',
        header=primeiro  # Escreve o cabeçalho apenas na primeira vez
    )
    primeiro = False

    print(f"Salvo dados para {date}")

print("Todos os resultados salvos incrementalmente em", csv_path)



#### The Brazilian Global Atmospheric Model (TQ0666L064 / Hybrid) #####

Forecast data available for reading between 20250602 and 20250612.

Surface variables: t2m, u10m, v10m, slp, psfc, precip
                   terrain, sbcape, sbcin, pw.
Level variables:   t, u, v, rh, g, omega.

levels (hPa): 1000  925  850  775  700  500  400  300  250
              200 150  100   70   50   30   20   10    3.

Frequency: every 6 hours [0, 6, 12, 18,...,168].

Iniciando a partir de 2023112300
Solicitando para 2023112300
Retrying pd.read_csv..
Retrying pd.read_csv..
Retrying pd.read_csv..
Salvo dados para 2023112300
Solicitando para 2023112400
Retrying pd.read_csv..
Error with http response 502
Retrying pycurl..
Error with http response 502
Retrying pycurl..
Salvo dados para 2023112400
Solicitando para 2023112500
Error with http response 502
Retrying pycurl..
Salvo dados para 2023112500
Solicitando para 2023112600
Retrying pd.read_csv..
Salvo dados para 2023112600
Solicitando para 2023112700
Retry